In [1]:
%load_ext autoreload
%autoreload 2

In [15]:
import Packages.CoNLLReader as reader
import numpy as np
from tqdm import tqdm
from Levenshtein import distance

In [16]:
text, data = reader.read_aida_yago_conll(
    "D:\\Sgmon\\Documents\\Magistrale\\TESI\\ClusteringAndLinking\\aida-yago2-dataset\\AIDA-YAGO2-dataset.tsv")
save = False
if save:
    text_file = open('text.txt', 'w')
    text_file.write(text)
    text_file.close()
data

,documents,tokens,indexes,tags,mentions,entities,wikidatas,numeric_codes,alpha_codes
0,1,EU,"(0, 2)",,,,,,
1,1,rejects,"(3, 10)",,,,,,
2,1,German,"(11, 17)",B,German,Germany,http://en.wikipedia.org/wiki/Germany,11867,/m/0345h
3,1,call,"(18, 22)",,,,,,
4,1,to,"(23, 25)",,,,,,
...,...,...,...,...,...,...,...,...,...
285584,1310,younger,"(1615647, 1615654)",,,,,,
285585,1310,brother,"(1615655, 1615662)",,,,,,
285586,1310,",","(1615663, 1615664)",,,,,,
285587,1310,Bobby,"(1615665, 1615670)",B,Bobby,Bobby_Charlton,http://en.wikipedia.org/wiki/Bobby_Charlton,4224,/m/01c8x


In [14]:
n_entities = sum([x is not '' for x in list(data['entities'])])
n_ass_ents = sum([x is not '' for x in list(data['numeric_codes'])])
n_tokens = sum([1 for x in list(data['entities'])])

In [5]:
print('{0:<35} {1:>10} '.format("Numero totale di entità:", n_entities))
print('{0:<35} {1:>10} '.format("Numero totale di tokens:", n_tokens))
print('{0:<35} {1:>10} '.format("1 entità ogni:", round(n_tokens / n_entities, 2)))

Numero totale di entità:                 27817 
Numero totale di tokens:                285589 
1 entità ogni:                           10.27 


In [6]:
ent_nums = {}
ents_data = data[data['entities'] != '']
entities = ents_data['entities'].values
unique_entities = np.unique(entities)
for uniqu_ent in tqdm(entities):
    ent_nums[uniqu_ent] = 0
for uniqu_ent in tqdm(entities):
    ent_nums[uniqu_ent] = ent_nums[uniqu_ent] + 1
ent_nums = dict(sorted(ent_nums.items(), key=lambda item: item[1], reverse=True))

100%|██████████| 27817/27817 [00:00<00:00, 1324459.42it/s]


In [7]:
ents_data = data[data['entities'] != '']
mentions = ents_data['mentions'].values

mention_to_entity = {}
for men, ent in set(list(zip(list(ents_data['mentions']), list(ents_data['entities'])))):
    mention_to_entity[men] = ent

### Golden standard

In [8]:
golden_standard = [mention_to_entity[x] for x in mentions]
golden_standard_dict = dict.fromkeys(golden_standard, 0)
for ent in golden_standard:
    golden_standard_dict[ent] = golden_standard_dict[ent] + 1

### Clustering by Levenshtein distance and DBSCAN

In [9]:
# Way with dbscan algorithm
import numpy as np
from sklearn.cluster import dbscan

clustering = False
if clustering:
    def lev_metric(x, y):
        i, j = int(x[0]), int(y[0])  # extract indices
        if len(mentions[i]) < 4:
            if mentions[i] == mentions[j]:
                return 0
            else:
                return distance(mentions[i].lower(), mentions[j].lower()) + 3
        else:
            return distance(mentions[i].lower(), mentions[j].lower())


    X = np.arange(len(mentions)).reshape(-1, 1)
    _, leven_cluster = dbscan(X, metric=lev_metric, eps=1, min_samples=1, n_jobs=-1)
    np.savetxt('db_cluster_levestein.txt', leven_cluster, delimiter=',')
else:
    leven_cluster = np.loadtxt("../aida-yago2-dataset/db_cluster_levestein_1.txt", dtype=np.int32)

In [16]:
# Now we create a dict for each cluster that contains entities and entities count
lev_cluster_dict = {}
for i, x in enumerate(leven_cluster):
    try:
        lev_cluster_dict[x].append(mentions[i])
    except:
        lev_cluster_dict[x] = [mentions[i]]
del lev_cluster_dict[-1]
for key in tqdm(lev_cluster_dict):
    cluster_list = [mention_to_entity[x] for x in lev_cluster_dict[key]]
    cluster_set = set(cluster_list)
    cluster_dict = dict.fromkeys(list(cluster_set), 0)
    for cluster in cluster_list:
        cluster_dict[cluster] = cluster_dict[cluster] + 1
    lev_cluster_dict[key] = cluster_dict

100%|██████████| 6058/6058 [00:00<00:00, 296852.55it/s]


### evaluation levestein

CEAFm precision

In [19]:
max_lev_cluster_list = []
for key in lev_cluster_dict.keys():
    max_key = max(lev_cluster_dict[key], key=lev_cluster_dict[key].get)
    max_lev_cluster_list.append((max_key, lev_cluster_dict[key][max_key]))
max_lev_cluster_list.sort(key=lambda x: x[1], reverse=True)
max_lev_cluster_dict = dict.fromkeys(entities, 0)
for key, val in max_lev_cluster_list:
    if max_lev_cluster_dict[key] == 0:
        max_lev_cluster_dict[key] = val

In [20]:
# CEAFm_levenshtein_precision
CEAFm_levenshtein_precision = sum([x for x in max_lev_cluster_dict.values()]) / ents_data.shape[0]
CEAFm_levenshtein_precision

0.84268612718841

CEAFm recall

In [21]:
# CEAFm_levenshtein_recall
CEAFm_levenshtein_recall = sum([x for x in max_lev_cluster_dict.values()]) / sum(
    [y for x in lev_cluster_dict.values() for y in x.values()])
CEAFm_levenshtein_recall

0.8429285483116976

In [22]:
# CEAFm_levenshtein_f1
CEAFm_levenshtein_f1 = (2 * (CEAFm_levenshtein_recall * CEAFm_levenshtein_precision)) / (
        CEAFm_levenshtein_precision + CEAFm_levenshtein_recall)
CEAFm_levenshtein_f1

0.842807320317837

In [27]:
# B-cubed - precision
bcubed_precision_num = 0
for gold_key in tqdm(golden_standard_dict.keys()):
    for lev_key in lev_cluster_dict.keys():
        try:
            bcubed_precision_num = bcubed_precision_num + (pow(lev_cluster_dict[lev_key][gold_key], 2) /
                                       sum([x for x in lev_cluster_dict[lev_key].values()]))
        except:
            pass
bcubed_precision = bcubed_precision_num/sum([y for x in lev_cluster_dict.values() for y in x.values()])
bcubed_precision

100%|██████████| 5283/5283 [00:13<00:00, 393.33it/s]


0.9152053912503401

In [28]:
# B-cubed - recall
bcubed_recall_num = 0
for gold_key in tqdm(golden_standard_dict.keys()):
    for lev_key in lev_cluster_dict.keys():
        try:
            bcubed_recall_num = bcubed_recall_num + (pow(lev_cluster_dict[lev_key][gold_key], 2) /
                                       golden_standard_dict[gold_key])
        except:
            pass
bcubed_recall = bcubed_recall_num/ents_data.shape[0]
bcubed_recall

100%|██████████| 5283/5283 [00:13<00:00, 397.67it/s]


0.8538846574245273

In [25]:
bcubed_f1 = (2 * (bcubed_recall * bcubed_precision)) / (
        bcubed_precision + bcubed_recall)
bcubed_f1

0.8834822654349819

In [26]:
sum([y for x in lev_cluster_dict.values() for y in x.values()])



27809